# 6. Advanced RAG


In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

## 6.2. 실습 준비


In [2]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
    langchain-community==0.3.0 GitPython==3.1.43 \
    langchain-chroma==0.1.4 tavily-python==0.5.0 \
    pydantic==2.9.1 numpy==1.25.2 langchain-cohere==0.3.0 rank-bm25==0.2.2

In [3]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

412


In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션 라이프사이클의 모든 단계를 간소화합니다. 주요 기능은 다음과 같습니다:\n\n1. **개발**: LangChain의 오픈 소스 구성 요소와 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 만들고, 스트리밍 및 인간-루프 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 애플리케이션을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 대형 언어 모델 및 관련 기술(예: 임베딩 모델 및 벡터 저장소)에 대한 표준 인터페이스를 구현하고 있으며, 수백 개의 제공업체와 통합됩니다. 이 프레임워크는 여러 오픈 소스 라이브러리로 구성되어 있으며, 다양한 구성 요소를 쉽게 결합하여 복잡한 애플리케이션을 구축할 수 있도록 지원합니다.'

## 6.3. 검색 쿼리의 기법


### HyDE（Hypothetical Document Embeddings）


In [6]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
다음 질문에 한 문장으로 답하세요.

질문: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [7]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 모든 단계, 즉 개발, 생산화, 배포를 간소화합니다. \n\n1. **개발**: LangChain의 오픈 소스 구성 요소와 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 구축할 수 있으며, 스트리밍 및 인간-루프 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 애플리케이션을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 대형 언어 모델 및 관련 기술(예: 임베딩 모델 및 벡터 저장소)에 대한 표준 인터페이스를 구현하고 있으며, 수백 개의 공급자와 통합됩니다. 이를 통해 개발자는 다양한 구성 요소를 쉽게 결합하고 복잡한 애플리케이션을 구축할 수 있습니다.'

### 복수 검색 쿼리 생성


In [8]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="검색 쿼리 목록")


query_generation_prompt = ChatPromptTemplate.from_template("""\
질문에 대해 벡터 데이터베이스에서 관련 문서를 검색하기 위한
3개의 서로 다른 검색 쿼리를 생성하세요.
거리 기반 유사성 검색의 한계를 극복하기 위해
사용자의 질문에 대해 여러 관점을 제공하는 것이 목표입니다.

질문: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [9]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 대형 언어 모델(LLM)을 기반으로 하는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 모든 단계, 즉 개발, 생산화, 배포를 간소화합니다.\n\n1. **개발**: LangChain의 오픈 소스 구성 요소와 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 구축하고, 스트리밍 및 인간-인-루프 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 애플리케이션을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 대형 언어 모델 및 관련 기술(예: 임베딩 모델 및 벡터 저장소)에 대한 표준 인터페이스를 구현하며, 수백 개의 공급자와 통합됩니다. 이 프레임워크는 여러 오픈 소스 라이브러리로 구성되어 있으며, 다양한 구성 요소를 조합하여 프로덕션 준비가 완료된 애플리케이션을 구축할 수 있는 오케스트레이션 프레임워크인 LangGraph를 포함하고 있습니다.'

## 6.4. 검색 후 기법


### RAG Fusion


In [10]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 각 문서의 콘텐츠(문자열)와 그 점수의 매핑을 저장하는 딕셔너리 준비
    content_score_mapping = {}

    # 검색 쿼리마다 반복
    for docs in retriever_outputs:
        # 검색 결과의 문서마다 반복
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 처음 등장한 콘텐츠인 경우 점수를 0으로 초기화
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (순위 + k)) 점수를 추가
            content_score_mapping[content] += 1 / (rank + k)

    # 점수가 큰 순서로 정렬
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [11]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 모든 단계, 즉 개발, 생산화, 배포를 간소화합니다.\n\n1. **개발**: LangChain의 오픈 소스 구성 요소와 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 만들고, 스트리밍 및 인간 개입 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 애플리케이션을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 대형 언어 모델 및 관련 기술(예: 임베딩 모델 및 벡터 저장소)에 대한 표준 인터페이스를 구현하고 있으며, 수백 개의 제공업체와 통합됩니다. 이를 통해 개발자는 다양한 구성 요소를 쉽게 결합하고 전환할 수 있습니다.'

### Cohere 리랭크 모델 사용 준비


In [12]:
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

In [13]:
!pip install langchain-cohere==0.3.0

### Cohere 리랭크 모델 도입


In [14]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document


def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)


rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션 라이프사이클의 모든 단계를 간소화합니다. 주요 기능은 다음과 같습니다:\n\n1. **개발**: LangChain의 오픈 소스 구성 요소와 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 만들고, 스트리밍 및 인간 개입 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 애플리케이션을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 대형 언어 모델 및 관련 기술(예: 임베딩 모델 및 벡터 저장소)에 대한 표준 인터페이스를 구현하고 있으며, 수백 개의 공급자와 통합됩니다. 이 프레임워크는 여러 오픈 소스 라이브러리로 구성되어 있으며, 각 구성 요소는 독립적으로 사용할 수 있습니다.'

## 6.5. 복수 Retriever 활용 기법


### LLM에 의한 라우팅


In [15]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [16]:
from enum import Enum


class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
질문에 답변하기 위한 적절한 Retriever를 선택하세요.

질문: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [17]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown route: {route}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [18]:
route_rag_chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션 라이프사이클의 모든 단계를 간소화합니다. 주요 기능은 다음과 같습니다:\n\n1. **개발**: LangChain의 오픈 소스 구성 요소와 제3자 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 만들고, 스트리밍 및 인간 개입 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 애플리케이션을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 대형 언어 모델 및 관련 기술(예: 임베딩 모델 및 벡터 저장소)에 대한 표준 인터페이스를 구현하고 있으며, 수백 개의 제공업체와 통합됩니다. 이 프레임워크는 여러 오픈 소스 라이브러리로 구성되어 있으며, 각 구성 요소는 특정 기능을 수행합니다. LangChain은 개발자가 애플리케이션을 쉽게 구축하고 관리할 수 있도록 돕는 다양한 도구와 리소스를 제공합니다.'

In [19]:
route_rag_chain.invoke("오늘 서울 날씨는?")

'오늘 서울의 날씨는 대체로 맑고, 기온은 오후 12시에 34°F(약 1°C)로 시작하여 오후 1시에 37°F(약 3°C), 오후 2시에 39°F(약 4°C)로 상승합니다. 오후 3시에는 40°F(약 4°C), 오후 4시에는 41°F(약 5°C)로 계속 상승하며, 오후 5시에는 39°F(약 4°C)로 떨어집니다. 바람은 서북서 방향에서 불며, 대기질은 나쁨으로 나타납니다. 전반적으로 화창한 날씨가 지속됩니다.'

### 하이브리드 검색 구현


In [20]:
!pip install rank-bm25==0.2.2

In [21]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [22]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [23]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 대형 언어 모델(LLM)을 기반으로 한 애플리케이션 개발을 위한 프레임워크입니다. 이 프레임워크는 LLM 애플리케이션의 모든 단계, 즉 개발, 생산화, 배포를 간소화합니다.\n\n1. **개발**: LangChain의 오픈 소스 구성 요소와 타사 통합을 사용하여 애플리케이션을 구축할 수 있습니다. LangGraph를 사용하여 상태를 유지하는 에이전트를 만들고, 스트리밍 및 인간 개입 지원을 제공합니다.\n\n2. **생산화**: LangSmith를 사용하여 애플리케이션을 검사, 모니터링 및 평가하여 지속적으로 최적화하고 자신 있게 배포할 수 있습니다.\n\n3. **배포**: LangGraph 애플리케이션을 프로덕션 준비가 완료된 API 및 어시스턴트로 변환할 수 있습니다.\n\nLangChain은 대형 언어 모델 및 관련 기술(예: 임베딩 모델 및 벡터 저장소)에 대한 표준 인터페이스를 구현하고 있으며, 수백 개의 제공업체와 통합됩니다. 이를 통해 개발자는 다양한 모델과 구성 요소를 쉽게 결합하여 복잡한 애플리케이션을 구축할 수 있습니다.'